In [ ]:
  !pip install datasets transformers
! pip install -U accelerate
! pip install -U transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install datasets

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML
from transformers import AutoTokenizer
import json

In [ ]:
model_checkpoint = "hfl/chinese-roberta-wwm-ext"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
##########
# 1. Load the data
##########
# Import the training data
with open('/content/drive/My Drive/NQuAD/NQuAD_final_train_7.5k.json', 'r', encoding='utf-8') as file:
        train_data = json.load(file)

# Import the testing data
with open('/content/drive/My Drive/NQuAD/NQuAD_final_test_1.5k.json', 'r', encoding='utf-8') as file:
        test_data = json.load(file)

In [ ]:
data_train[0]

{'context': '立凱-KY(5227)12月營收9,348萬元，年減8.74%，係8月與五龍合作以來首呈年減，立凱表示衰退係因中國面臨電動車補助政策空窗等不確定因素，拉貨放緩之故，而1月、2月雖已排除政策空窗因素，但仍受農曆年工作天數減少干擾，3月營收料可恢復正常，全年營收則在五龍補助電池芯研發金帶動下，估計可有5~10%年增空間。立凱-KY為磷酸鐵鋰電池正極材料大廠，目前營收比重100%為電池正極材料；立凱-KY已於2016年與香港上市公司五龍電動車集團及五龍動力(378.HK)三方合作，目前五龍持股立凱約21%，未來將持續深化雙方在電動車產業的上下游合作佈局，共同爭取全球及中國電動車市場發展所帶來的巨大商機。立凱12月單月營收9,348萬元，年減8.74%，係與五龍合作以來首度營收呈現年減，立凱指出營收年減係因當時中國電動車新補助政策尚未出爐，客戶呈觀望態度下單保守所致，而1月營收展望上，雖政策面觀望因素已經排除，但又因有農曆年干擾，工作天數較少，預期營收水準仍與12月相當。回顧立凱去年營運成績，在去年8月26日與正式與五龍簽署合作後，立凱下半年營收開始認列五龍提供的電池芯研發金，全年營收13.18億元，年增2成。至於在獲利上，在電動車部門併入五龍旗下，立凱費用亦有減輕，第三季單季獲利翻正，稅後淨利7,900萬元，單季EPS 0.43元，前三季稅後淨損1,900萬元，EPS -0.11元，法人評估立凱第四季仍可保持獲利，去年全年獲利表現可望轉正。展望今年後續表現，日前中國政策預計將向外國電池業者砍價，立凱因下游電池客戶均為中國國內廠商，第一大客戶係力神、第二大客戶則為五龍，兩者總計約佔營收六成，是以不致影響到立凱後續表現；至於全年展望，法人評估立凱3月起即可走出先前補助政策空窗與農曆年影響，全年營收在五龍支持電池芯研發金進帳帶動下，將有5~10%的向上空間。',
 'ans0': '立凱今年Q1較淡 全年營收仍看增5~10%',
 'ans1': '立凱今年Q1較淡 全年營收仍看增5~12%',
 'ans2': '立凱今年Q1較淡 全年營收仍看增5~8%',
 'ans3': '立凱今年Q1較淡 全年營收仍看增5~8.74%',
 'ans': 0}

In [ ]:
from datasets import Dataset, DatasetDict, ClassLabel, Features, Value

# 定义数据集特征，为每个answer提供单独的字段
features = Features({
    'context': Value('string'),
    'answer0': Value('string'),
    'answer1': Value('string'),
    'answer2': Value('string'),
    'answer3': Value('string'),
    'label': ClassLabel(num_classes=4, names=['0', '1', '2', '3'])
})

# 准备数据
prepared_train_data = {
    'context': [],
    'answer0': [],
    'answer1': [],
    'answer2': [],
    'answer3': [],
    'label': []
}

prepared_test_data = {
    'context': [],
    'answer0': [],
    'answer1': [],
    'answer2': [],
    'answer3': [],
    'label': []
}

for item in data_train:
    prepared_train_data['context'].append(item['context'].strip())
    prepared_train_data['label'].append(item['ans'])
    for i in range(4):
        prepared_train_data[f'answer{i}'].append(item[f'ans{i}'].strip())


for item in data_test:
    prepared_test_data['context'].append(item['context'].strip())
    prepared_test_data['label'].append(item['ans'])
    for i in range(4):
        prepared_test_data[f'answer{i}'].append(item[f'ans{i}'].strip())

# 创建单个 Dataset
train_data = Dataset.from_dict(prepared_train_data, features=features)
test_data = Dataset.from_dict(prepared_test_data, features=features)

In [ ]:
datasets = DatasetDict({
    'train': train_data,
    'test': test_data
})

In [ ]:
datasets['train'] = datasets['train'].select(range(7500))
datasets['test'] = datasets['test'].select(range(1500))

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=3):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
def show_one(example):
    print(f"Context: {example['context']}")
    print(f"  A - {example['answer0']}")
    print(f"  B - {example['answer1']}")
    print(f"  C - {example['answer2']}")
    print(f"  D - {example['answer3']}")
    print(f"\nGround truth: option {['A', 'B', 'C', 'D'][example['label']]}")

show_one(datasets["train"][0])

# Context: 宏達電（2498）昨(21)日股東會上，董事長王雪紅應允小股東請求，自掏腰包贈送股東會議程結束前完成報到程序且人在會場的小股東（不含公司內部股東），每人一台宏達電旗艦手機新HTC One 32G手機、單機市價約2.19萬元，創下台灣股東會史上單價最高的贈禮。同時，宏達電董監改選，前台積電（2330）總經理蔡力行當選新任董事，被外界認為是跟台積電聯手抗韓，針對三星電子的科技競爭而進行的結盟。據統計，當日符合條件股東數約100多位，依單機2.19萬元計算，王雪紅大手筆送給在場關心宏達電營運的小股東們逾200多萬元大禮。經過各家媒體近兩日報導，「宏達電」、「HTC」、「新HTC One」在網路上討論的熱烈程度又升高了，達到品牌與知名度的提升效果，宏達電行銷功力可謂更上層樓。宏達電今年的股東會也全面進行董監事改選，當選董事名單包括王雪紅、陳文琦、卓火土、蔡力行、David Bruce Yoffie；獨立董事為林振國、Josef Felder；新任監察人為威智投資(股)公司、朱黃傑。除了蔡力行是新赴任的宏達電董事外，其餘本次當選董事、監察人都為續任。蔡力行是台積電（2330）前總經理、現任台積電太陽能與固態照明董事長；因此外界也多解讀，在台積電董事長張忠謀公開多次盛讚宏達電手機產品，並倡議聯發科（2454）、鴻海（2317）、台積電與宏達電在各自的半導體IC、面板、晶圓產業、消費性手機領域一同對付三星電子；蔡力行昨日獲選出任宏達電董事一職，可謂是台灣科技廠聯手抗韓的最新發展。
#   A - 王雪紅贈股東100G新One，創台股東會單價最高贈品
#   B - 王雪紅贈股東2.19G新One，創台股東會單價最高贈品
#   C - 王雪紅贈股東21G新One，創台股東會單價最高贈品
#   D - 王雪紅贈股東32G新One，創台股東會單價最高贈品

# Ground truth: option D

Context: 立凱-KY(5227)12月營收9,348萬元，年減8.74%，係8月與五龍合作以來首呈年減，立凱表示衰退係因中國面臨電動車補助政策空窗等不確定因素，拉貨放緩之故，而1月、2月雖已排除政策空窗因素，但仍受農曆年工作天數減少干擾，3月營收料可恢復正常，全年營收則在五龍補助電池芯研發金帶動下，估計可有5~10%年增空間。立凱-KY為磷酸鐵鋰電池正極材料大廠，目前營收比重100%為電池正極材料；立凱-KY已於2016年與香港上市公司五龍電動車集團及五龍動力(378.HK)三方合作，目前五龍持股立凱約21%，未來將持續深化雙方在電動車產業的上下游合作佈局，共同爭取全球及中國電動車市場發展所帶來的巨大商機。立凱12月單月營收9,348萬元，年減8.74%，係與五龍合作以來首度營收呈現年減，立凱指出營收年減係因當時中國電動車新補助政策尚未出爐，客戶呈觀望態度下單保守所致，而1月營收展望上，雖政策面觀望因素已經排除，但又因有農曆年干擾，工作天數較少，預期營收水準仍與12月相當。回顧立凱去年營運成績，在去年8月26日與正式與五龍簽署合作後，立凱下半年營收開始認列五龍提供的電池芯研發金，全年營收13.18億元，年增2成。至於在獲利上，在電動車部門併入五龍旗下，立凱費用亦有減輕，第三季單季獲利翻正，稅後淨利7,900萬元，單季EPS 0.43元，前三季稅後淨損1,900萬元，EPS -0.11元，法人評估立凱第四季仍可保持獲利，去年全年獲利表現可望轉正。展望今年後續表現，日前中國政策預計將向外國電池業者砍價，立凱因下游電池客戶均為中國國內廠商，第一大客戶係力神、第二大客戶則為五龍，兩者總計約佔營收六成，是以不致影響到立凱後續表現；至於全年展望，法人評估立凱3月起即可走出先前補助政策空窗與農曆年影響，全年營收在五龍支持電池芯研發金進帳帶動下，將有5~10%的向上空間。
  A - 立凱今年Q1較淡 全年營收仍看增5~10%
  B - 立凱今年Q1較淡 全年營收仍看增5~12%
  C - 立凱今年Q1較淡 全年營收仍看增5~8%
  D - 立凱今年Q1較淡 全年營收仍看增5~8.74%

Ground truth: option A


In [ ]:
from transformers import AutoTokenizer
model_checkpoint = "hfl/chinese-roberta-wwm-ext"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True, padding=True, truncation=True, max_length=512)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
print(datasets['train'][0]['answer0'])

立凱今年Q1較淡 全年營收仍看增5~10%


In [ ]:
ans_names = ['answer0', 'answer1', 'answer2', 'answer3']

def preprocess_function(examples):

    # 预处理输入tokenizer的输入
    # Repeat each first sentence four times to go with the four possibilities of second sentences.
    first_sentences = [[context] * 4 for context in examples["context"]]# 构造和备选项个数相同的问题句，也是tokenizer的第一个句子
    # for each in first_sentences:
    #   for per in each:
    #        print(per)
    # print(first_sentences)
    # print(f'len of first_sentence: {len(first_sentences)}\n')

    # 构造所有的second sentences，每个second sentence是一个备选答案
    second_sentences = [
        [examples[ans][i] for ans in ans_names] for i in range(len(examples["context"]))
    ]
    # print()
    # for each in second_sentences:
    #   for per in each:
    #     print(per)
    # # print(first_sentences)
    # print(f'len of second_sentence: {len(second_sentences)}\n')


    # # Flatten everything
    first_sentences = sum(first_sentences, []) # 合并成一个列表方便tokenizer一次性处理
    # print(f'first_sentences_flatted: {first_sentences}')
    # print(f'{len(first_sentences)}\n')
    second_sentences = sum(second_sentences, [])# 合并成一个列表方便tokenizer一次性处理
    # print(f'_sentences_flatted: {second_sentences}')
    # print(f'{len(second_sentences)}\n')

    # Tokenize
    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True,  max_length=512)

    # Un-flatten
    # 转化成每个样本（一个样本中包括了四个k=[问题1,问题1,问题1,问题1],v=[备选项1,备选项2,备选项3,备选项4]）

    return {k: [v[i:i+4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}

In [ ]:
examples = datasets["train"][:2]
features = preprocess_function(examples)

In [ ]:
idx = 0
[tokenizer.decode(features["input_ids"][idx][i]) for i in range(4)]

['[CLS] 立 凱 - ky ( 5227 ) 12 月 營 收 9, 348 萬 元 ， 年 減 8. 74 % ， 係 8 月 與 五 龍 合 作 以 來 首 呈 年 減 ， 立 凱 表 示 衰 退 係 因 中 國 面 臨 電 動 車 補 助 政 策 空 窗 等 不 確 定 因 素 ， 拉 貨 放 緩 之 故 ， 而 1 月 、 2 月 雖 已 排 除 政 策 空 窗 因 素 ， 但 仍 受 農 曆 年 工 作 天 數 減 少 干 擾 ， 3 月 營 收 料 可 恢 復 正 常 ， 全 年 營 收 則 在 五 龍 補 助 電 池 芯 研 發 金 帶 動 下 ， 估 計 可 有 5 ~ 10 % 年 增 空 間 。 立 凱 - ky 為 磷 酸 鐵 鋰 電 池 正 極 材 料 大 廠 ， 目 前 營 收 比 重 100 % 為 電 池 正 極 材 料 ； 立 凱 - ky 已 於 2016 年 與 香 港 上 市 公 司 五 龍 電 動 車 集 團 及 五 龍 動 力 ( 378. hk ) 三 方 合 作 ， 目 前 五 龍 持 股 立 凱 約 21 % ， 未 來 將 持 續 深 化 雙 方 在 電 動 車 產 業 的 上 下 游 合 作 佈 局 ， 共 同 爭 取 全 球 及 中 國 電 動 車 市 場 發 展 所 帶 來 的 巨 大 商 機 。 立 凱 12 月 單 月 營 收 9, 348 萬 元 ， 年 減 8. 74 % ， 係 與 五 龍 合 作 以 來 首 度 營 收 呈 現 年 減 ， 立 凱 指 出 營 收 年 減 係 因 當 時 中 國 電 動 車 新 補 助 政 策 尚 未 出 爐 ， 客 戶 呈 觀 望 態 度 下 單 保 守 所 致 ， 而 1 月 營 收 展 望 上 ， 雖 政 策 面 觀 望 因 素 已 經 排 除 ， 但 又 因 有 農 曆 年 干 擾 ， 工 作 天 數 較 少 ， 預 期 營 收 水 準 仍 與 12 月 相 當 。 回 顧 立 凱 去 年 營 運 成 績 ， 在 去 年 8 月 26 日 與 正 式 與 五 龍 簽 署 合 作 後 ， 立 凱 下 半 年 營 收 開 始 認 列 五 龍 提 供 的 電 池 芯 研 發 金 ， 全 年 營 收 13. 18 億 元 ， 年 增 2 成 。 至 於 在 獲 

In [ ]:
tokenized_datasets = datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForMultipleChoice
model = AutoModelForMultipleChoice.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at hfl/chinese-roberta-wwm-ext and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import accelerate
print(accelerate.__version__)

0.29.3


In [ ]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

In [ ]:
@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        #features:[{'attention_mask':[[],[],...],'input_ids':[[],[],...,'label':_},{'attention_mask':[[],[],...],'input_ids':[[],[],...,'label':_}]
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features] #将label单独弹出，features:[{'attention_mask':[[],[],...],'input_ids':[[],[],...]},{'attention_mask':[[],[],...],'input_ids':[[],[],...]}]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])

        #feature:{'attention_mask':[[],[],...],'input_ids':[[],[],...]}
        #flattened_features:[[{'attention_mask':[],'input_ids':[]},{},{},{}],[]....]
        flattened_features = [[{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features]
        #flattened_features:[{'attention_mask':[],'input_ids':[]},{},{},{},{}....]
        flattened_features = sum(flattened_features, [])

        # batch: {'attention_mask':[[],[],[],[],[],[],...],'input_ids':[[],[],[],[],[],[],...]}
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        # Un-flatten
        # batch: {'attention_mask':[[[],[],[],[]],[[],[],[],[]],[...],...],'input_ids':[[[],[],[],[]],[[],[],[],[]],[...],...]}
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        # Add back labels
        # batch: {'attention_mask':[[[],[],[],[]],[[],[],[],[]],[...],...],'input_ids':[[[],[],[],[]],[[],[],[],[]],[...],...],'label':[]}
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [ ]:
accepted_keys = ["input_ids", "attention_mask", "label"]
features = [{k: v for k, v in tokenized_datasets["train"][i].items() if k in accepted_keys} for i in range(10)]
batch = DataCollatorForMultipleChoice(tokenizer)(features)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
[tokenizer.decode(batch["input_ids"][8][i].tolist()) for i in range(4)]

In [ ]:
show_one(datasets["train"][8])
#    Context: Someone walks over to the radio.
#      A - Someone hands her another phone.
#      B - Someone takes the drink, then holds it.
#      C - Someone looks off then looks at someone.
#      D - Someone stares blearily down at the floor.
#
#    Ground truth: option D

Context: PC需求疲弱，連接器廠商嘉澤(3533)首季營收僅較去年同期持平，法人估計，因少了庫存評價對毛利率的正向挹注，首季毛利率可能較去年同期走降，加上費用及匯兌損失影響，每股稅後盈餘將落在1-1.5元。但首季營運可望落底，第二季營收將呈個位數增長，全年營收力拼微增，每股稅後盈餘挑戰維持8元水準。而隨股價淨值比修正至1.03倍的歷史低檔，長期投資價值浮現。嘉澤目前營收比重中，PC應用(含DT/NB，並以DT占大宗)超過七成，另有二成多來自伺服器。如果以產品來看，單價較高的CPU Socket貢獻較大，其次則為DDR系列，其他還有I/O類產品。去年營收82.08億元，稅後盈餘7.96億元，每股稅後盈餘8.52元。而因PC應用偏淡，又少了去年同期伺服器應用的重複下單(Overbooking)，嘉澤首季營收僅較去年同期持平，法人也估計，毛利率會略低於去年平均，且不如去年同期，加上營業費用提高，業外可能提列2、3千萬元的匯兌損失，單季每股稅後盈餘將落在1-1.5元。展望後市，法人認為，PC供應鏈仍偏淡，不過，對嘉澤而言，有機會略優於整體產業表現的原因有二，一是隨DDR4與DDR3價差拉近，PC轉換速度有望加快，年底有望取代DDR3成為主流，有利DDR系列產品平均單價提高。另外，因PC供應鏈走向大者恆大，嘉澤今年在特定客戶滲透率有機會提升，帶動表現優於產業平均。而在伺服器方面，DDR4轉換率已高，因此今年嘉澤成長動能將轉至CPU Socket於品牌供應鏈的滲透增長。據了解，公司已取得二家品牌廠的認證，雖貢獻緩步增長，但下半年效益將逐步放大。法人估計，在工作天數回升與伺服器應用增長下，嘉澤第二季營收可望季增個位數，伺服器貢獻年增率將轉正，若台幣兌美元走穩，獲利增幅會高於營收。而下半年隨PC旺季加溫，營收將重回兩位數的季增動能，全年營收力拼較去年微增，每股稅後盈餘挑戰維持8元水準。
  A - 嘉澤首季營運落底；股淨比修正至約1倍水準
  B - 嘉澤首季營運落底；股淨比修正至約1.03倍水準
  C - 嘉澤首季營運落底；股淨比修正至約2倍水準
  D - 嘉澤首季營運落底；股淨比修正至約3倍水準

Ground truth: option A


In [ ]:
import numpy as np
from datasets import load_metric
def compute_metrics(eval_predictions):
    predictions, label_ids = eval_predictions
    preds = np.argmax(predictions, axis=1)
    return {"accuracy": (preds == label_ids).astype(np.float32).mean().item()}

In [ ]:
!export TF_FORCE_GPU_ALLOW_GROWTH=true

In [ ]:
# import gc
# for obj in gc.get_objects():
#     if torch.is_tensor(obj) and obj.is_cuda:
#         del obj
# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
task='ner'
batch_size = 5

from transformers import  TrainingArguments

In [ ]:
args = TrainingArguments(
    "my_model_output",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [ ]:
from transformers import  Trainer
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer),
    compute_metrics=compute_metrics,
)

In [ ]:
from accelerate import Accelerator
accelerator = Accelerator()

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.717500,0.589467,0.768000
2,0.503600,0.516098,0.823333
3,0.386200,0.594669,0.818000


TrainOutput(global_step=4500, training_loss=0.6137298278808594, metrics={'train_runtime': 10315.2968, 'train_samples_per_second': 2.181, 'train_steps_per_second': 0.436, 'total_flos': 2.367399603357084e+16, 'train_loss': 0.6137298278808594, 'epoch': 3.0})